In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from optuna.samplers import TPESampler
import optuna
import pickle
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
from tqdm import tqdm
# min max scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from utils.util_functions import import_data
from sklearn import preprocessing
from utils.group_ts_split import GroupTimeSeriesSplit, PurgedGroupTimeSeriesSplit

In [21]:
df_train = import_data('data/extra_train.csv')
# drop brackets from column names
df_train.columns = df_train.columns.str.replace('[', '', regex=False)
df_train.columns = df_train.columns.str.replace(']', '', regex=False)
df_train.columns = df_train.columns.str.replace(' ', '_', regex=False)
df_train

Memory usage of dataframe is 8.66 MB
Memory usage after optimization is: 2.29 MB
Decreased by 73.6%


,Month,Hour,cons_target,Solar_Generation_W/kW,Outdoor_Drybulb_Temperature_C,Relative_Humidity_%,Diffuse_Solar_Radiation_W/m2,Direct_Solar_Radiation_W/m2,day_year,number,...,net_target,net+1,net-23,net_target+1,net_target-23,net_target_diff,diffuse_solar_radiation+1,direct_solar_radiation+1,relative_humidity+1,drybulb_temp+1
0,8,0,0.441406,0.000000,19.406250,81.0,0.000000,0.000000,23,1,...,0.617188,1.411133,2.275391,0.431885,0.506836,-0.029327,0.000000,0.000,81.0,20.000000
1,8,1,0.170776,0.000000,20.000000,81.0,0.000000,0.000000,24,1,...,0.431885,0.979492,0.851074,0.394531,0.383301,-0.185547,0.000000,0.000,87.0,19.406250
2,8,2,0.116333,0.000000,19.406250,87.0,0.000000,0.000000,25,1,...,0.394531,0.900879,0.834473,0.387695,0.382080,-0.037354,0.000000,0.000,87.0,19.406250
3,8,3,0.106445,0.000000,19.406250,87.0,0.000000,0.000000,26,1,...,0.387695,0.922363,0.838379,0.389648,0.382324,-0.006783,0.000000,0.000,90.0,19.406250
4,8,4,0.109131,0.000000,19.406250,90.0,0.000000,0.000000,27,1,...,0.389648,0.910645,1.478516,0.388428,0.437744,0.001856,0.000000,0.000,90.0,18.906250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43675,7,19,0.802734,31.171875,21.093750,81.0,5.390625,5.667969,8754,5,...,0.862305,2.714844,3.462891,0.720703,0.814941,0.090393,3.294922,4.875,84.0,20.593750
43676,7,20,0.552734,4.070312,20.593750,84.0,3.294922,4.875000,8755,5,...,0.720703,0.936035,0.457275,0.497803,0.437744,-0.141235,0.000000,0.000,81.0,21.093750
43677,7,21,0.189453,0.000000,21.093750,81.0,0.000000,0.000000,8756,5,...,0.497803,2.154297,0.714844,0.650879,0.469971,-0.223145,0.000000,0.000,79.0,21.703125
43678,7,22,0.436279,0.000000,21.703125,79.0,0.000000,0.000000,8757,5,...,0.650879,2.316406,1.659180,0.670898,0.588379,0.152954,0.000000,0.000,76.0,21.296875


In [22]:
df_train.columns

Index(['Month', 'Hour', 'cons_target', 'Solar_Generation_W/kW',
       'Outdoor_Drybulb_Temperature_C', 'Relative_Humidity_%',
       'Diffuse_Solar_Radiation_W/m2', 'Direct_Solar_Radiation_W/m2',
       'day_year', 'number', 'hour_x', 'hour_y', 'month_x', 'month_y', 'net',
       'net_target', 'net+1', 'net-23', 'net_target+1', 'net_target-23',
       'net_target_diff', 'diffuse_solar_radiation+1',
       'direct_solar_radiation+1', 'relative_humidity+1', 'drybulb_temp+1'],
      dtype='object')

In [23]:
X = df_train.drop(['number', 'net', 'day_year'], axis=1)
# for j in range(1,25):
#     X = X.drop([f'net+{j}', f'net_target+{j}', f'net-{j-1}'], axis=1)
X_temp = X.drop(['net+1', 'net_target+1', 'net-23', 'Solar_Generation_W/kW',
       'Outdoor_Drybulb_Temperature_C', 'Relative_Humidity_%',
       'Diffuse_Solar_Radiation_W/m2', 'Direct_Solar_Radiation_W/m2', 'net_target_diff','cons_target', 'net_target'], axis=1)
features = X_temp.columns
categoricals = ['Month','Hour']
print(categoricals)
y = df_train['net_target+1']
groups = df_train['day_year'].values
X_temp

['Month', 'Hour']


,Month,Hour,hour_x,hour_y,month_x,month_y,net_target-23,diffuse_solar_radiation+1,direct_solar_radiation+1,relative_humidity+1,drybulb_temp+1
0,8,0,1.000000,0.000000,-0.500000,-0.866211,0.506836,0.000000,0.000,81.0,20.000000
1,8,1,0.965820,0.258789,-0.500000,-0.866211,0.383301,0.000000,0.000,87.0,19.406250
2,8,2,0.866211,0.500000,-0.500000,-0.866211,0.382080,0.000000,0.000,87.0,19.406250
3,8,3,0.707031,0.707031,-0.500000,-0.866211,0.382324,0.000000,0.000,90.0,19.406250
4,8,4,0.500000,0.866211,-0.500000,-0.866211,0.437744,0.000000,0.000,90.0,18.906250
...,...,...,...,...,...,...,...,...,...,...,...
43675,7,19,0.258789,-0.965820,-0.866211,-0.500000,0.814941,3.294922,4.875,84.0,20.593750
43676,7,20,0.500000,-0.866211,-0.866211,-0.500000,0.437744,0.000000,0.000,81.0,21.093750
43677,7,21,0.707031,-0.707031,-0.866211,-0.500000,0.469971,0.000000,0.000,79.0,21.703125
43678,7,22,0.866211,-0.500000,-0.866211,-0.500000,0.588379,0.000000,0.000,76.0,21.296875


In [24]:
evals_result = {}
sampler = TPESampler(seed = 123)

def objective(trial):
    params = {
        'objective': 'regression_l1',
        'metric': 'mae',
        'random_state': 48,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_categorical("max_depth", [-1, 5, 10, 15, 20, 25]), 
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 100, step=10),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
    }
        
    # 6-fold time-series cross validation
    tfold = GroupTimeSeriesSplit()
    mae_list = []
    for _, (train_index, test_index) in tqdm(enumerate(tfold.split(X_temp, y, groups))):
        X_train = X_temp.iloc[train_index]
        X_val = X_temp.iloc[test_index]
        y_train = y[train_index]
        y_val = y[test_index]        
        train_dataset = lgb.Dataset(X_train, 
                            y_train, 
                            categorical_feature = categoricals,
                            free_raw_data=False)
        val_dataset = lgb.Dataset(X_val, 
                                y_val,
                                categorical_feature = categoricals,
                                free_raw_data=False)
        # train LGBM2
        model = lgb.train(params = params,
                          train_set = train_dataset, 
                          valid_sets = [val_dataset],
                          callbacks=[lgb.early_stopping(100)]
                         )
        preds = model.predict(val_dataset.get_data())
        mae = mean_absolute_error(y_val, preds)
    mae_list.append(mae)
    trial.set_user_attr(key="best_booster", value=model)
    print(f'\n average loss: {np.mean(mae_list)}')
    return np.mean(mae_list)

def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_booster", value=trial.user_attrs["best_booster"])

In [25]:
study_model = optuna.create_study(direction = 'minimize', sampler = sampler)
study_model.optimize(objective, n_trials = 50, callbacks=[callback])
best_model=study_model.user_attrs["best_booster"]

[I 2023-03-12 09:51:31,968] A new study created in memory with name: no-name-b59abefa-bbed-43bb-8b6b-29c27402dfc7
0it [00:00, ?it/s]/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
1it [00:00,  2.42it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000174 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 869
[LightGBM] [Info] Number of data points in the train set: 7280, number of used features: 11
[LightGBM] [Info] Start training from score 0.412598
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 955
[LightGBM] [Info] Number of data points in the train set: 14560, number of used features: 11
[LightGBM] [Info] Start training from score 0.408691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

2it [00:01,  1.13it/s]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Did not meet early stopping. Best iteration is:
[39]	valid_0's l1: 0.0595643


/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000585 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1011
[LightGBM] [Info] Number of data points in the train set: 36400, number of used features: 11
[LightGBM] [Info] Start training from score 0.410889
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

5it [00:16,  3.31s/it]
[I 2023-03-12 10:01:35,157] Trial 49 finished with value: 0.07488165336784283 and parameters: {'learning_rate': 0.2251089126965405, 'num_leaves': 1060, 'max_depth': 25, 'min_data_in_leaf': 30, 'lambda_l1': 25, 'lambda_l2': 30}. Best is trial 45 with value: 0.07336283792980403.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Did not meet early stopping. Best iteration is:
[22]	valid_0's l1: 0.0748817

 average loss: 0.07488165336784283


In [26]:
joblib.dump(best_model, 'models/lgb_24step.pkl')

['models/lgb_24step.pkl']